Cargo los datos sobre temperaturas durante el año 2024 de Barcelona desde la estación de Drassanes que tiene un criterio de temperatura a nivel ciudad.

In [1]:
import pandas as pd

# Cargo los archivos CSV
df1 = pd.read_csv('temp_012024_062024.csv')
df2 = pd.read_csv('temp_072024_122024.csv')

# Concateno los dataFrames por filas
df_temp_2024 = pd.concat([df1, df2], axis=0, ignore_index=True)

# Muestro el dataFrame combinado
df_temp_2024.head()

,fecha,indicativo,nombre,provincia,altitud,tmed,prec,tmin,horatmin,tmax,horatmax,hrMedia,hrMax,horaHrMax,hrMin,horaHrMin
0,2024-01-01,0201X,"BARCELONA, DRASSANES",BARCELONA,11,"11,8","0,0","8,8",23:52,"14,7",14:45,68.0,76.0,05:00,61.0,10:50
1,2024-01-02,0201X,"BARCELONA, DRASSANES",BARCELONA,11,"11,6","0,0","7,9",03:53,"15,3",15:59,46.0,66.0,00:00,27.0,16:00
2,2024-01-03,0201X,"BARCELONA, DRASSANES",BARCELONA,11,"15,4","0,0","12,6",07:23,"18,2",12:22,61.0,71.0,07:40,50.0,01:30
3,2024-01-04,0201X,"BARCELONA, DRASSANES",BARCELONA,11,"14,2","0,4","11,6",07:02,"16,9",13:18,79.0,85.0,08:40,65.0,00:00
4,2024-01-05,0201X,"BARCELONA, DRASSANES",BARCELONA,11,"11,6","8,8","9,6",Varias,"13,6",12:46,79.0,89.0,04:30,67.0,13:30


El dataframe está compuesto por 366 filas y 16 columnas

In [2]:
df_temp_2024.shape

(366, 16)

Hay columnas que no contemplo utilizar por lo que decido eliminarlas para aligerar el dataframe.

In [3]:
df_temp_2024 = df_temp_2024.drop(columns=['horaHrMax', 'horaHrMin', 'hrMedia', 'hrMax', 'hrMin', 'indicativo', 'nombre', 'altitud','horatmin','horatmax'])

Reviso las características de cada columna, observo varios detalles sobre ellas que considero tratar:

* `fecha` es del tipo `object` y el formato debería ser `datetime`
* Hay varias columas que aparecen como tipo `object` al utiizar una coma como decimal y las transformaré como float con punto.


In [4]:
df_temp_2024.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 366 entries, 0 to 365
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   fecha      366 non-null    object
 1   provincia  366 non-null    object
 2   tmed       365 non-null    object
 3   prec       366 non-null    object
 4   tmin       365 non-null    object
 5   tmax       365 non-null    object
dtypes: object(6)
memory usage: 17.3+ KB


Realizo la transformación de la columna `fecha` al tipo `datetime`

In [5]:
df_temp_2024['fecha']= pd.to_datetime(df_temp_2024['fecha'], format='%Y-%m-%d')
print(df_temp_2024.dtypes)

fecha        datetime64[ns]
provincia            object
tmed                 object
prec                 object
tmin                 object
tmax                 object
dtype: object


Transformo las columnas 'tmed', 'prec', 'tmin', 'tmax' al tipo float, modificando el símbolo de coma por punto.

In [6]:
columnas_a_float = ['tmed', 'prec', 'tmin', 'tmax']

# Reemplazo las comas por puntos para que pueda realizarse el cambio al tipo float
for col in columnas_a_float:
    df_temp_2024[col] = df_temp_2024[col].str.replace(',', '.')  

df_temp_2024[columnas_a_float] = df_temp_2024[columnas_a_float].astype(float)

print(df_temp_2024.dtypes)

fecha        datetime64[ns]
provincia            object
tmed                float64
prec                float64
tmin                float64
tmax                float64
dtype: object


In [7]:
df_temp_2024.isnull().sum()

fecha        0
provincia    0
tmed         1
prec         0
tmin         1
tmax         1
dtype: int64

Analizo el valor nulo de la variable `tmed`, observo que aparecen varias columnas en ese registro(337) con valores faltantes en las columnas mostradas en la revisión de los valores faltantes en el dataframe.

In [8]:
nulos_tmed = df_temp_2024['tmed'].isnull()
df_temp_2024[nulos_tmed]

,fecha,provincia,tmed,prec,tmin,tmax
337,2024-12-03,BARCELONA,NaN,0.6,NaN,NaN


La idea de la imputación en este caso es mantener la continuidad de la serie. Imputo los valores faltantes utilizando una función, que en caso de que encuentre un valor nulo en una columna, buscará el valor correspondiente en la fila anterior (día anterior). Si el valor anterior existe, se utiliza para reemplazar el valor nulo. 

In [9]:

# Ordeno el DataFrame por fecha para asegurar la correcta imputación
df_temp_2024 = df_temp_2024.sort_values('fecha')

# Selecciono las columnas a imputar 
columnas_a_imputar = ['tmed', 'tmin', 'tmax']

# Se itera sobre las filas y las columnas a imputar
for index, row in df_temp_2024.iterrows():
    for columna in columnas_a_imputar:
        if pd.isna(row[columna]):
            # Buscar el valor del día anterior en la misma columna
            valor_anterior = df_temp_2024.loc[index - 1, columna]
            # Imputar si el valor anterior existe
            if not pd.isna(valor_anterior):
                df_temp_2024.loc[index, columna] = valor_anterior

Compruebo que los valores ha sido imputados totalmente.

In [10]:
df_temp_2024.isnull().sum()

fecha        0
provincia    0
tmed         0
prec         0
tmin         0
tmax         0
dtype: int64

In [11]:
df_temp_2024.head()

,fecha,provincia,tmed,prec,tmin,tmax
0,2024-01-01,BARCELONA,11.8,0.0,8.8,14.7
1,2024-01-02,BARCELONA,11.6,0.0,7.9,15.3
2,2024-01-03,BARCELONA,15.4,0.0,12.6,18.2
3,2024-01-04,BARCELONA,14.2,0.4,11.6,16.9
4,2024-01-05,BARCELONA,11.6,8.8,9.6,13.6


- Renombro la columna `provincia` por `ciudad`.

In [12]:
df_temp_2024= df_temp_2024.rename(columns={'provincia': 'ciudad'})
df_temp_2024= df_temp_2024[['fecha', 'ciudad', 'tmin', 'tmax', 'tmed', 'prec']]
df_temp_2024.head()


,fecha,ciudad,tmin,tmax,tmed,prec
0,2024-01-01,BARCELONA,8.8,14.7,11.8,0.0
1,2024-01-02,BARCELONA,7.9,15.3,11.6,0.0
2,2024-01-03,BARCELONA,12.6,18.2,15.4,0.0
3,2024-01-04,BARCELONA,11.6,16.9,14.2,0.4
4,2024-01-05,BARCELONA,9.6,13.6,11.6,8.8


Exporto el dataframe a un archivo de formato `csv`

In [13]:
df_temp_2024.to_csv('temp_bcn_2024.csv', index= False)

Resumen del dataframe creado:

- Se eliminaron las columnas 'indicativo', 'nombre','hrMedia', 'hrMax', 'horaHrMax', 'hrMin', 'horaHrMin', 'altitud', 'dir', 'velmedia', 'racha', 'horaracha', 'horatmin', 'horatmax'.
- Se realizaron transformaciones de tipo en las columnas 'tmed', 'prec', 'tmin', 'tmax'.
- Se transformó la columna 'fecha' a tipo datetime
- Se trataron los valores nulos mediante la imputación del registro del día anterior al faltante, buscando mantener la continuidad de la serie. No hay valores faltantes.
- Se renombró la columna 'provincia' por 'ciudad'.
- Se exporto el dataframe a un archivo en formato csv.